In [1]:
import subprocess
from astropy.table import Table
import os
import csv
#in the furture make packages and import common functions
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from pyraf import gwm
from astropy.io import ascii
import numpy as np
from astropy.stats import sigma_clip
from astropy.io import fits
from astropy.io import ascii


In [2]:
from pyraf import iraf
from iraf import obsutil
from iraf import psfmeasure
from iraf import phot
from iraf import daofind
from iraf import daophot
from iraf import psf
from iraf import artdata
from iraf import mkobjects
from iraf import starlist

In [3]:
imagename = '/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits'
diffImage = '/home/mj1e16/Simages/diff_44_1.fits'
SImage= '/home/mj1e16/Simages/testsimage.fits'
ext = 44

In [4]:
outDir = '/home/mj1e16/iraf/outputs/' # for iraf output files ONLY
imageDir = '/home/mj1e16/Simages/irafImages/' # for images generated during process ONLY
simageDir = '/home/mj1e16/Simages/' # directory for simulated images
irafDir = '/home/mj1e16/iraf/' #Directory containing iraf installation

In [35]:
def mkobjectspluspsf(imagename,diffimage,diffPlusSim,ext,section,minMag,maxMag,ccd,Xmax=1000,Xmin=0,Ymax=1000,Ymin=1000,thresh=50000,nsegs=5,goodpsf='',spatialDist='uniform',zeropoint=25.):
    
    daoout = outDir+'{}_{}daofindone'.format(ext,section)
    photout = outDir+'{}_{}photone'.format(ext,section)
    psfout = outDir+'{}_{}psfone'.format(ext,section)
    groupout = outDir+'{}_{}groupfileone'.format(ext,section)
    opstout = outDir+'{}_{}optsone'.format(ext,section)
    starfieldsect = '/home/mj1e16/iraf/starlists/starfield_{}_{}.dat'.format(mag,section)
    imseg = imageDir+'imageseg{}.fits'.format(section)
    
    hdu = fits.open(imagename)
    print(imagename)
    image = hdu[ext].section[Xmin:Xmax,Ymin:Ymax]
    hdu = fits.PrimaryHDU(image)
    hdu.writeto(imseg,clobber=True)
    
#     with open(starfieldsect,'w') as f:
#         f.write("")
    
    daofind(imseg,output=daoout,sigma=1.0,threshold=thresh)
    phot(imseg,skyfile=daoout,coords=daoout,output=photout,interactive='no')
    psf(imseg,photfile=daoout,pstfile=photout,psfimage=psfout,opstfile=opstout,groupfile=groupout,interactive='no')
#     if ccd == 44:
#         starlist(starfieldsect,nstars=1000/nsegs,xmin=Xmin,xmax=Xmax,ymin=Ymin,ymax=Ymax,spatial=spatialDist,
#                  minmag=minMag,maxmag=maxMag) #rememeber to switch off when doing other images
    try:
        mkobjects(diffimage,output=diffPlusSim,objects=starfieldsect,gain=110,rdnoise=127,star=psfout+'.fits')
        
    except:
        try:
            #subprocess.call(['rm','/home/mj1e16/Simages/testsimage*'])
            mkobjects(diffimage,output=diffPlusSim,objects=starfieldsect,gain=110,rdnoise=127,star=goodpsf)
            print('Waring: Section {} with bad PSF, {} used inplace'.format(section,goodpsf))
        except:
            print('ERROR: PSF for Section {} should be made manually'.format(section))
    
    return psfout+'.fits'

In [36]:
def findPSFandMkObjects(imageName,diffImage,SImage,extension,minMag,maxMag,ccd,imageshape=[1015., 1085.],pixelsize=3.98):
    
    print(imageName)
    
    os.chdir(irafDir)
    
    xlength = imageshape[1]
    ylength = imageshape[0]
    nsegs = 4
    
    xsegment = xlength/nsegs
    ysegment = ylength/nsegs
    PSF = []
    goodpsf=''
    for xsegs in range(nsegs):
        lowx = int(xsegs*xsegment)+15
        highx = int(lowx+xsegment)+15
        psf = []
        for ysegs in range(0,nsegs):
            lowy = int(ysegs*ysegment)+15
            highy = int(lowy+ysegment)+15
            
            
            # stars are being lost from images in xseg=0
            SImage= imageDir+'testsimage_{}_{}_{}.fits'.format(xsegs,ysegs,abs(minMag))
            
            goodpsf = mkobjectspluspsf(imageName,diffImage,SImage,extension,'{}_{}'.format(xsegs,ysegs),minMag,maxMag,ccd,Xmax=highx,Xmin=lowx,Ymax=highy,Ymin=lowy,nsegs=nsegs**2,goodpsf=goodpsf)
            
            b = diffImage.find('{}_alt'.format(minMag))
            if b == -1:
                diffImage = diffImage[:-5]+'{}_alt.fits'.format(minMag)
            try:
                hdu = fits.open(SImage) #,ignore_missing_end=True)
                imdata = hdu[0].data
                hdu = fits.PrimaryHDU(imdata)
                hdu.writeto(diffImage,clobber=True)
                print('find me')
                print(SImage)
                print(diffImage)
            except:
                print('bad at section {} {}'.format(xsegs,ysegs))

            

            
    return 'complete!'

In [37]:
def cleanDirectories(dirlist):
    # clean iraf output and input images
    # make sure to have no needed files in these directories
    for directory in dirlist:
        dirfiles = os.listdir(directory)
        for eachFile in dirfiles:
            #print('rm',directory+eachFile)
            subprocess.call(['rm',directory+eachFile])

In [38]:
def stitchStarList(nsegs,mag):
    with open(outDir+'starfield_-7_0_0.dat') as f:
        lines = f.readlines()
    bigstring = ['# X_POS Y_POS MAG\n']
    bigstring.extend(lines[0:18])
    for x in range(nsegs):
        for y in range(nsegs):
            with open(outDir+'starfield_{}_{}_{}.dat'.format(mag,x,y)) as f:
                lines = f.readlines()
            bigstring.extend(lines[18:])


    with open('/home/mj1e16/keplerPhotometry/starlistFull_{}.dat'.format(mag),'w') as f:
        f.write(''.join(bigstring))


In [39]:
cleanDirectories([outDir,imageDir]) # cleans iraf output files, will remove all files from these directories

In [40]:
run = 'not complete'
for mag in range(-7,0):
    cleanDirectories([outDir,imageDir]) # cleans iraf output files, will remove all files from these directories
    while run == 'not complete':
        try:
            run = findPSFandMkObjects(imagename,diffImage,SImage,ext,mag,(mag+1),ext)
            run = 'complete'
        except:
            print('break at ',x)
            pass

/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pi

bad at section 0 0
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pixels
File/list of aperture radii in 

bad at section 0 1
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pixels
File/list of aperture radii in 

bad at section 0 2
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pixels
File/list of aperture radii in 

bad at section 0 3
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pixels
File/list of aperture radii in 

...

bad at section 2 3
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pixels
File/list of aperture radii in 

bad at section 3 0
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pixels
File/list of aperture radii in 

bad at section 3 1
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pixels
File/list of aperture radii in 

bad at section 3 2
/data/mj1e16/kepler/cal/kplr2009114174833_ffi-cal.fits

FWHM of features in scale units (2.5) (CR or value): 	New FWHM of features: 2.5 scale units  2.5 pixels
Standard deviation of background in counts (1.) (CR or value): 	New standard deviation of background: 1. counts
Detection threshold in sigma (50000.) (CR or value): 	New detection threshold: 50000. sigma 50000. counts
Minimum good data value (INDEF) (CR or value): 	New minimum good data value: INDEF counts
Maximum good data value (INDEF) (CR or value): 	New maximum good data value: INDEF counts


Centering algorithm (none) (CR or value): 	New centering algorithm: none
Sky fitting algorithm (mode) (CR or value): 	Sky fitting algorithm: mode
Inner radius of sky annulus in scale units (10.) (CR or value): 	New inner radius of sky annulus: 10. scale units 10. pixels
Width of the sky annulus in scale units (10.) (CR or value): 	New width of the sky annulus: 10. scale units 10. pixels
File/list of aperture radii in 

bad at section 3 3


In [25]:
for mag in range(-7,0):
    stitchStarList(4,mag)

In [17]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'global color=lightgreen\nimage\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},5)\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase + str(tables)+'.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
        print(fileName)


In [15]:
data = ascii.read('/home/mj1e16/keplerPhotometry/starlistFull.dat')

In [16]:
makeDS9RegFile([data],'/home/mj1e16/Simages/teststarlist','iraf')

/home/mj1e16/Simages/teststarlist0.reg
['44_2_0photone', 'logfile_63_x2_y7', '44photone', 'logfile_9_x8_y13', 'logfile_63_x13_y6', 'starfield_0_0.dat', 'logfile_44_x0_y10', 'logfile_9_x9_y3', '44_3_3photone', 'logfile_44_x13_y2', 'mkObjects.ipynb', 'logfile_44_x13_y12', 'logfile_63_x1_y10', 'logfile_63_x0_y10', 'test19.fits', 'logfile_44_x8_y4', 'login.cl', 'logfile_44_x6_y7', '44_0_4groupfileone', 'logfile_9_x13_y11', '44_0_0photone', 'logfile_9_x9_y2', 'removeStarfield.sh', 'logfile_63_x7_y9', 'logfile_9_x5_y7', 'logfile_63_x13_y13', 'logfile_63_x9_y12', 'test4.fits', 'logfile_63_x6_y5', 'logfile_9_x12_y1', 'imaginary1000.fits', 'logfile_63_x4_y3', 'logfile_9_x12_y2', 'test6.fits', '44_4_1daofindone', 'logfile_44_x6_y9', 'logfile_9_x12_y9', 'logfile_9_x14_y7', 'starfield_1_3.dat', 'logfile_9_x11_y9', 'aaa.sh', 'logfile_44_x8_y9', 'logfile_9_x7_y11', 'logfile_44_x14_y8', 'logfile_63_x7_y2', 'logfile_63_x2_y0', 'logfile_63_x6_y12', 'logfile_9_x0_y9', 'logfile_9_x8_y12', 'logfile_44_x6_

In [2]:
for x in range(-7,0):
    print(x)

-7
-6
-5
-4
-3
-2
-1
